# About: 検証環境の構築--コンテナのセキュリティアップデート

---

Moodleコンテナのパッケージを更新する。

## 概要

Moodleコンテナのパッケージをアップデートする場合の検証環境を構築します。

![検証環境](images/moodle-121-01.png)

### グループ名の指定

このNotebookの操作対象となる UnitGroup名を指定します。

In [ ]:
# (例)
# ugroup_name = 'Moodle'

ugroup_name =

#### チェック

指定された `UnitGroup`名に対応する group_varsファイルが存在していることを確認します。エラーになる場合は、指定したUnitGroup名が正しくないと考えられます。

In [ ]:
from pathlib import Path
if not (Path('group_vars') / (ugroup_name + '.yml')).exists():
    raise RuntimeError(f"ERROR: not exists {ugroup_name + '.yml'}")

UnitGroupに属する VCノードに対して Ansible で操作できることを確認します。

In [ ]:
!ansible {ugroup_name} -m ping
!ansible {ugroup_name} -b -a 'whoami'

### VCCアクセストークンの入力

VCCにアクセスするためのトークンを入力します。

In [ ]:
from getpass import getpass
vcc_access_token = getpass()

#### チェック

入力されたアクセストークンが正しいことを、VCCにアクセスして確認します。

In [ ]:
from vcpsdk.vcpsdk import VcpSDK
vcp = VcpSDK(vcc_access_token)

### 準備

これまでに他のNotebookで設定したパラメータを読み込む処理などを行います。



group_varsファイルに保存されているパラメータを読み込みます。

In [ ]:
%run scripts/group.py
gvars = load_group_vars(ugroup_name)

VCCのVault サーバにアクセスする際に必要となるパラメータを環境変数に設定します。

In [ ]:
import os
os.environ['VAULT_ADDR'] = vcp.vcc_info()['vault_url']
os.environ['VAULT_TOKEN'] = vcc_access_token

## パラメータの設定

対象となるコンテナ名や、検証環境にアクセスできる条件などを設定します。

アップデート対象となるコンテナ名を指定します。現在実行中のコンテナを確認するためにコンテナ名の一覧を表示します。

In [ ]:
!ansible {ugroup_name} -a 'chdir=/opt/moodle docker-compose ps --services' \
    | grep {gvars['project_tag']}

アップデート対象となるコンテナ名を指定してください。

In [ ]:
# (例)
# update_container_name = 'moodle-0'

update_container_name =

ここで構築する環境では、アクセス元のIPアドレスによって検証環境と運用環境との切り替えを行います。
検証環境を表示させることにするIPアドレスを指定してください。

In [ ]:
# (例)
# update_test_client = '192.168.10.0/24'
# update_test_client = '192.168.10.10'

update_test_client =

現在の運用環境と、検証環境を区別するために用いるタグの値（文字列）を指定してください。

> 各環境のコンテナ名や論理ボリューム名はタグ含む名前が付けられます。そのため、コンテナ名や論理ボリューム名から、どの環境に属するコンテナ、論理ボリュームであるかを判別することができます。

In [ ]:
# (例)
# update_project_tag = '001'
# update_project_tag = '20200101XXXXXX'

update_project_tag =

タグに指定する値は、シリアルナンバーまたはタイムスタンプからなる文字列を付けることをお勧めしますが、任意の英数字からなる文字列を指定することができます。

指定されたタグが既存のものと重複していないことをチェックします。

In [ ]:
if (update_project_tag == gvars['project_tag']
    or update_project_tag in
    [x['project_tag']
     for x in gvars.get('previous_info_list', [])
     if 'project_tag' in x]):
    raise RuntimeError("既に使用したタグ名です")

指定されたパラメータを group_vars に保存します。

In [ ]:
update_container_target = update_container_name.split('-')[0]
update_group_vars(
    ugroup_name,
    update_project_tag=update_project_tag,
    update_container_target=update_container_target,
)

## 検証環境の作成

現在運用中の環境を複製して、検証環境を作成します。

![検証環境の作成](images/moodle-121-02.png)

### 論理ボリュームの複製

現在、運用環境で使用中の論理ボリュームを複製するためにスナップショットを作成します。

In [ ]:
project_tag = gvars['project_tag']

# LVとVGとの対応
lv_map = {
    'moodle': 'moodle',
    'php': 'moodle',
    'db': 'db',
}

# スナップショットを作成する
for name, vg in lv_map.items():
    lv = f'{project_tag}_{name}'
    snapshot = f'{update_project_tag}_{name}_test'
    print(f'vg={vg} lv={lv} snapshot={snapshot}')
    !ansible {ugroup_name} -b -m lvol -a \
        'vg={vg} lv={lv} snapshot={snapshot} opts="-kn"'

スナップショットが作成されたことを確認するために、論理ボリュームの一覧を表示します。

In [ ]:
!ansible {ugroup_name} -b -a 'lvs -S "lv_attr=~V" -O lv_name'

作成したスナップショットをマウント可能にするために、論理ボリュームのアクティベートとUUIDの再作成を行います。

In [ ]:
for name, vg in lv_map.items():
    snapshot = f'{update_project_tag}_{name}_test'
    dev = f'/dev/mapper/{vg}-{snapshot}'
    print(f'dev={dev}')
    !ansible {ugroup_name} -b -m shell -a \
        'lvchange -ay {vg}/{snapshot} && \
         mount -o nouuid {dev} /mnt && \
         umount /mnt && \
         xfs_admin -U generate {dev}'

スナップショットがアクティベートされたことを確認するために、論理ボリュームの一覧を表示します。

In [ ]:
!ansible {ugroup_name} -b -a 'lvs -S "lv_attr=~V" -O lv_name'

作成したボリュームをマウントします。

In [ ]:
mount_point = {
    'moodle': f'/opt/moodle/moodletest-{update_project_tag}/data/moodledata',
    'php': f'/opt/moodle/moodletest-{update_project_tag}/data/php',
    'db': f'/opt/moodle/dbtest-{update_project_tag}/data',
}

for name, vg in lv_map.items():
    snapshot = f'{update_project_tag}_{name}_test'
    dev = f'/dev/mapper/{vg}-{snapshot}'
    print(f'path={mount_point[name]} src={dev}')
    !ansible {ugroup_name} -b -m mount -a \
        'path={mount_point[name]} src={dev} fstype=xfs state=mounted'

### 設定ファイルの複製

運用環境の設定ファイルをコピーして検証環境の設定ファイルを作成します。

In [ ]:
for name in ['moodle', 'db']:
    src = f'{name}-{project_tag}/conf'
    dst = f'{name}test-{update_project_tag}'
    print(f'cp -a {src} {dst}')
    !ansible {ugroup_name} -b -a \
        'chdir=/opt/moodle cp -a {src} {dst}'

コピーされた設定ファイルの一覧を確認します。

In [ ]:
target_dir = [f'{x}test-{update_project_tag}/conf'
              for x in ['moodle', 'db']]
!ansible {ugroup_name} -b -a \
    'chdir=/opt/moodle tree {" ".join(target_dir)}'

### コンテナイメージ

運用環境の更新対象となるコンテナから、検証環境用のベースとなるコンテナイメージを作成します。

In [ ]:
target_image = f'local/{update_container_target}:{update_project_tag}-test'
print(target_image)
!ansible {ugroup_name} -a \
    'docker commit {update_container_name} {target_image}'

コンテナイメージが作成されたことを確認します。　

In [ ]:
!ansible {ugroup_name} -a 'docker images {target_image}'

### docker-compose.yml の更新

`docker-compose.yml`に検証環境のコンテナを追加します。

次のセルを実行すると検証環境用コンテナを追加した `docker-compose.yml` をローカル環境に作成し更新前との差分を表示します。また最後に表示されたリンクから更新後の `docker-compose.yml` を編集することもできます。

In [ ]:
%run scripts/edit_conf.py
update_docker_compose(ugroup_name)

更新した`docker-compose.yml`をVCノードに配置します。

In [ ]:
upload_docker_compose(ugroup_name)

## 検証環境の起動

検証環境のコンテナを起動します。

![検証環境の起動](images/moodle-121-03.png)

### 検証環境コンテナの起動
検証環境のコンテナを起動します。

現在のコンテナの実行状態を確認します。　

In [ ]:
!ansible {ugroup_name} -a 'chdir=/opt/moodle \
    docker-compose ps'

検証用のコンテナを起動します。

In [ ]:
!ansible {ugroup_name} -a 'chdir=/opt/moodle \
    docker-compose up -d'

検証用コンテナを起動した後の、コンテナの実行状態を確認します。

In [ ]:
!ansible {ugroup_name} -a 'chdir=/opt/moodle \
    docker-compose ps'

### パッケージのアップデート

検証環境のアップデート対象のコンテナでパッケージをアップデートします。

In [ ]:
target_container = f'{update_container_target}test-{update_project_tag}'
print(target_container)

!ansible {ugroup_name} -a 'chdir=/opt/moodle \
    docker-compose exec -T {target_container} bash -c \
    "( type yum && yum update -y ) || \
     ( type apt-get && apt-get update && apt-get -y upgrade )"'

パッケージのアップデート結果を反映させるためにコンテナの再起動を行います。

In [ ]:
!ansible {ugroup_name} -a 'chdir=/opt/moodle \
    docker-compose restart {target_container}'

### アップデートされたパッケージの確認

運用環境と検証環境のパッケージの一覧を取得して比較することで、更新されたパッケージを確認します。

運用環境のコンテナのパッケージ一覧を取得します。

In [ ]:
current_container = f'{update_container_target}-{gvars["project_tag"]}'
package_current = !ansible {ugroup_name} -m shell -a 'chdir=/opt/moodle \
    docker-compose exec -T {current_container} \
    bash -c "( type rpm > /dev/null 2>&1 && rpm -qa ) || \
             ( type dpkg > /dev/null 2>&1 && dpkg -l )" | sort'

検証環境のコンテナのパッケージ一覧を取得します。

In [ ]:
package_new = !ansible {ugroup_name} -m shell -a 'chdir=/opt/moodle \
    docker-compose exec -T {target_container} \
    bash -c "( type rpm > /dev/null 2>&1 && rpm -qa ) || \
             ( type dpkg > /dev/null 2>&1 && dpkg -l )" | sort'

パッケージ一覧の差分を確認します。

In [ ]:
from difflib import unified_diff

for line in unified_diff(package_current, package_new, fromfile=current_container,
                         tofile=target_container):
    print(line)

### リバースプロキシの設定

アクセス元のIPアドレスによって運用環境と検証環境を切り替えるようにリバースプロキシを設定します。

現在のリバースプロキシの設定ファイルを確認します。

In [ ]:
!ansible {ugroup_name} -a 'cat /opt/moodle/proxy/conf/moodle-proxy.conf'

次のセルを実行すると運用環境と検証環境を切り替えるように記述したApache HTTP Serverの設定ファイル
`moodle-proxy.conf`をローカル環境に作成します。作成後に現在の設定ファイルとの差分を表示します。
また、ローカル環境に生成した `moodle-proxy.conf` を編集するためのリンクを最後に表示します。

In [ ]:
%run scripts/edit_conf.py
update_proxy_conf(
    ugroup_name,
    extra_vars={'update_test_client': update_test_client})

ローカル環境の変更後の `moodle-proxy.conf` をVCノードに配置します。
その後、設定ファイルの記述内容を反映させるためにリバースプロキシコンテナを再起動します。

In [ ]:
apply_proxy_conf(ugroup_name)

再起動後のコンテナの状態を確認します。`proxy`コンテナの状態が `Up` になっていることを確認してください。

In [ ]:
!ansible {ugroup_name} -a 'chdir=/opt/moodle \
    docker-compose ps'

## Moodle を利用できることを確認

検証用のMoodle環境にアクセスして正しく動作していることを確認します。

次のセルを実行するとMoodleのアドレスが表示されます。

In [ ]:
from IPython.core.display import HTML
HTML(u'<a href="{0}/admin/index.php" target="_blank">{0}/admin/index.php</a>'.format(gvars['moodle_url']))